Подключение Google Disk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Подключение необходимых библиотек

In [ ]:
import dataset
import bs4 as bs
import urllib.request
import re
import nltk
from gensim.models import Word2Vec, KeyedVectors
from gensim.corpora import Dictionary
nltk.download('punkt')
nltk.download('stopwords')
import os
import pymorphy2

Подготовка текста Воздушного Кодекса

In [ ]:
article_text = ''
with open(os.path.join('/content/drive/MyDrive', 'NLP_work', 'example', 'txt', 'voz_kod', 'voz_kod', 'voz_kod.txt'), 'r', encoding='utf-8') as r:
    for line in r:
        article_text += line
processed_article = article_text.lower()
processed_article = re.sub('[^а-яА-Я]', ' ', processed_article)
processed_article = re.sub(r'\s+', ' ', processed_article)

Подкготовка набора данных из слов ВК

In [ ]:
# all_sentences = nltk.sent_tokenize(processed_article)
# all_words = [nltk.word_tokenize(sent) for sent in all_sentences]
all_words = nltk.word_tokenize(processed_article)
# print(len(all_words[0]))
# print(len(alll_words))
print(all_words)
# print('=======================================================================')
# print(alll_words)

In [ ]:
v1 = word2vec.wv["меры"]
v2 = word2vec.wv['экипаж']
print(v1)

In [ ]:
sim_words = word2vec.wv.most_similar("правила")
print(sim_words)

In [ ]:
print(word2vec.wv.similar_by_vector(v1 + v2))

In [ ]:
word2vec.train([["пилот", "пилотесса", "мужчина", "женщина"]], total_examples=4, epochs=1)

In [ ]:
import matplotlib.pyplot as plt
# print(word2vec.wv["кодекс"])
vec_voc = {k: word2vec.wv[k] for k in vocabulary.keys()}
print(len(vec_voc.values()), len(vec_voc.keys()))
# print(list(vec_voc.values()))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.manifold import TSNE
def tsne_scatterplot(model, word, list_names):
    """Plot in seaborn the results from the t-SNE dimensionality reduction 
    algorithm of the vectors of a query word,
    its list of most similar words, and a list of words."""
    vectors_words = [model.wv.get_vector(word)]
    word_labels = [word]
    color_list = ['red']
    close_words = model.wv.most_similar(word)
    for wrd_score in close_words:
        wrd_vector = model.wv.get_vector(wrd_score[0])
        vectors_words.append(wrd_vector)
        word_labels.append(wrd_score[0])
        color_list.append('blue')
    # adds the vector for each of the words from list_names to the array
    for wrd in list_names:
        wrd_vector = model.wv.get_vector(wrd)
        vectors_words.append(wrd_vector)
        word_labels.append(wrd)
        color_list.append('green')
    # t-SNE reduction
    Y = (TSNE(n_components=2, random_state=0, perplexity=15, init="pca")
        .fit_transform(vectors_words))
    # Sets everything up to plot
    df = pd.DataFrame({"x": [x for x in Y[:, 0]],
                    "y": [y for y in Y[:, 1]],
                    "words": word_labels,
                    "color": color_list})
    fig, _ = plt.subplots()
    fig.set_size_inches(9, 9)
    # Basic plot
    p1 = sns.regplot(data=df,
                    x="x",
                    y="y",
                    fit_reg=False,
                    marker="o",
                    scatter_kws={"s": 40,
                                "facecolors": df["color"]}
    )
    # Adds annotations one by one with a loop
#     for line in range(0, df.shape[0]):
#         p1.text(df["x"][line],
#                 df["y"][line],
#                 " " + df["words"][line].title(),
#                 horizontalalignment="left",
#                 verticalalignment="bottom", size="medium",
#                 color=df["color"][line],
#                 weight="normal"
#         ).set_size(12)
    plt.xlim(Y[:, 0].min()-50, Y[:, 0].max()+50)
    plt.ylim(Y[:, 1].min()-50, Y[:, 1].max()+50)
    plt.title('t-SNE visualization for {}'.format(word.title()))

def tsne_plot_common(model, list_words):
    vectors_words = []
    word_labels = []
    color_list = []
    for wrd in list_words:
        wrd_vector = model.wv.get_vector(wrd)
        vectors_words.append(wrd_vector)
        word_labels.append(wrd)
        color_list.append('blue')
     # t-SNE reduction
    Y = (TSNE(n_components=2, random_state=0, perplexity=15, init="pca")
        .fit_transform(vectors_words))
    # Sets everything up to plot
    df = pd.DataFrame({"x": [x for x in Y[:, 0]],
                    "y": [y for y in Y[:, 1]],
                    "words": word_labels,
                    "color": color_list})
    fig, _ = plt.subplots()
    fig.set_size_inches(9, 9)
    # Basic plot
    p1 = sns.regplot(data=df,
                    x="x",
                    y="y",
                    fit_reg=False,
                    marker="o",
                    scatter_kws={"s": 40,
                                "facecolors": df["color"]}
    )
    plt.xlim(Y[:, 0].min()-50, Y[:, 0].max()+50)
    plt.ylim(Y[:, 1].min()-50, Y[:, 1].max()+50)
    plt.title('t-SNE visualization')

In [ ]:
# tsne_scatterplot(word2vec, "аэродром", ["аэропорт", "судно", "пилот", "пассажир", "экипаж", "федеральный"])
# tsne_scatterplot(word2vec, "экипаж", vec_voc.keys())
tsne_plot_common(word2vec, vec_voc.keys())
plt.show()